# PKH Dashboard - Data Processing Pipeline (Colab)

Notebook ini menjelaskan **tahapan pengolahan data dari awal hingga menjadi tabel processed** seperti yang dipakai dashboard saat ini.

## Input file yang dibutuhkan (nama file harus sama)
- `bps-od_17058_persentase_penduduk_miskin__kabupatenkota_data.csv`
- `dinsos-od_20731_jml_penerima_bantuan_program_keluarga_harapan_pkh__v2_data.csv`
- `bps-od_16425_jumlah_penduduk_miskin_berdasarkan_kabupatenkota_data.csv`
- `bps-od_17112_jumlah_penduduk_miskin_berdasarkan_daerah_v9_data.csv`

Output (processed):
- `dim_kabupaten.csv`
- `fact_pkh.csv`
- `fact_kemiskinan_persen.csv`
- `fact_kemiskinan_abs.csv`
- `fact_kemiskinan_kategori.csv`


In [ ]:
# Setup lokasi kerja
import os
from pathlib import Path

BASE_DIR = Path("/content")
RAW_DIR = BASE_DIR / "data" / "raw"
PROCESSED_DIR = BASE_DIR / "data" / "processed"

RAW_DIR.mkdir(parents=True, exist_ok=True)
PROCESSED_DIR.mkdir(parents=True, exist_ok=True)

print("RAW_DIR:", RAW_DIR)
print("PROCESSED_DIR:", PROCESSED_DIR)


## Upload file CSV
Gunakan `files.upload()` lalu pindahkan file ke folder `RAW_DIR`.
Pastikan **nama file sama persis** dengan daftar di atas.

In [ ]:
from google.colab import files
uploaded = files.upload()

for name in uploaded.keys():
    target = RAW_DIR / name
    with open(target, "wb") as f:
        f.write(uploaded[name])

print("Files in RAW_DIR:")
for path in RAW_DIR.iterdir():
    print("-", path.name)


## Load & validasi kolom
Validasi memastikan setiap file memiliki kolom wajib sesuai pipeline di backend.

In [ ]:
import pandas as pd

DATASET_FILES = {
    "kemiskinan_persen": "bps-od_17058_persentase_penduduk_miskin__kabupatenkota_data.csv",
    "pkh": "dinsos-od_20731_jml_penerima_bantuan_program_keluarga_harapan_pkh__v2_data.csv",
    "kemiskinan_abs": "bps-od_16425_jumlah_penduduk_miskin_berdasarkan_kabupatenkota_data.csv",
    "kemiskinan_kategori": "bps-od_17112_jumlah_penduduk_miskin_berdasarkan_daerah_v9_data.csv",
}

REQUIRED_COLUMNS = {
    "kemiskinan_persen": {
        "kode_provinsi",
        "nama_provinsi",
        "kode_kabupaten_kota",
        "nama_kabupaten_kota",
        "persentase_penduduk_miskin",
        "tahun",
    },
    "pkh": {
        "kode_provinsi",
        "nama_provinsi",
        "kode_kabupaten_kota",
        "nama_kabupaten_kota",
        "jumlah_penerima_manfaat",
        "tahun",
    },
    "kemiskinan_abs": {
        "kode_provinsi",
        "nama_provinsi",
        "kode_kabupaten_kota",
        "nama_kabupaten_kota",
        "jumlah_penduduk_miskin",
        "tahun",
    },
    "kemiskinan_kategori": {
        "kode_provinsi",
        "nama_provinsi",
        "kategori_daerah",
        "periode_bulan",
        "jumlah_penduduk",
        "tahun",
    },
}

datasets = {}
for key, filename in DATASET_FILES.items():
    path = RAW_DIR / filename
    if not path.exists():
        raise FileNotFoundError(f"Missing dataset: {path}")
    datasets[key] = pd.read_csv(path)

def validate_dataset(df, key):
    required = REQUIRED_COLUMNS[key]
    missing = required - set(df.columns)
    if missing:
        raise ValueError(f"Missing columns for {key}: {sorted(missing)}")

for key, df in datasets.items():
    validate_dataset(df, key)
    print(key, "rows:", len(df), "cols:", len(df.columns))


## Normalisasi teks + filter Jawa Barat
Samakan penulisan nama kabupaten/kota & provinsi, lalu filter hanya provinsi Jawa Barat.

In [ ]:
PROV_CODE_JABAR = 32
PROV_NAME_JABAR = "JAWA BARAT"

def normalize_common(df):
    df = df.copy()
    if "nama_kabupaten_kota" in df.columns:
        df["nama_kabupaten_kota"] = (
            df["nama_kabupaten_kota"].astype(str).str.upper().str.replace(r"\s+", " ", regex=True).str.strip()
        )
    if "nama_provinsi" in df.columns:
        df["nama_provinsi"] = df["nama_provinsi"].astype(str).str.upper().str.strip()
    return df

def filter_jabar(df):
    if "kode_provinsi" in df.columns:
        df = df[df["kode_provinsi"] == PROV_CODE_JABAR]
    if "nama_provinsi" in df.columns:
        df = df[df["nama_provinsi"] == PROV_NAME_JABAR]
    return df

for key, df in datasets.items():
    df = normalize_common(df)
    df = filter_jabar(df)
    datasets[key] = df

for key, df in datasets.items():
    print(key, "rows after filter:", len(df))


## Coerce numeric + filter tahun
Pastikan kolom numerik dalam format angka dan batasi tahun (default 2017-2024).

In [ ]:
START_YEAR = 2017
END_YEAR = 2024

def coerce_numeric(df, cols):
    df = df.copy()
    for col in cols:
        if col in df.columns:
            df[col] = pd.to_numeric(df[col], errors="coerce")
    return df

persen = coerce_numeric(datasets["kemiskinan_persen"], ["kode_kabupaten_kota", "tahun", "persentase_penduduk_miskin"])
pkh = coerce_numeric(datasets["pkh"], ["kode_kabupaten_kota", "tahun", "jumlah_penerima_manfaat"])
abs_miskin = coerce_numeric(datasets["kemiskinan_abs"], ["kode_kabupaten_kota", "tahun", "jumlah_penduduk_miskin"])
kategori = coerce_numeric(datasets["kemiskinan_kategori"], ["tahun", "jumlah_penduduk"])

persen = persen[(persen["tahun"] >= START_YEAR) & (persen["tahun"] <= END_YEAR)]
pkh = pkh[(pkh["tahun"] >= START_YEAR) & (pkh["tahun"] <= END_YEAR)]
abs_miskin = abs_miskin[(abs_miskin["tahun"] >= START_YEAR) & (abs_miskin["tahun"] <= END_YEAR)]


## Build tabel dimensi & fakta + simpan
Menghasilkan tabel processed sesuai struktur dashboard.

In [ ]:
dim_kabupaten = (
    pd.concat(
        [
            persen[["kode_kabupaten_kota", "nama_kabupaten_kota", "kode_provinsi", "nama_provinsi"]],
            pkh[["kode_kabupaten_kota", "nama_kabupaten_kota", "kode_provinsi", "nama_provinsi"]],
        ],
        ignore_index=True,
    )
    .dropna(subset=["kode_kabupaten_kota"])
    .drop_duplicates(subset=["kode_kabupaten_kota"])
    .sort_values("nama_kabupaten_kota")
    .reset_index(drop=True)
)

fact_pkh = pkh[["tahun", "kode_kabupaten_kota", "nama_kabupaten_kota", "jumlah_penerima_manfaat"]]
fact_persen = persen[["tahun", "kode_kabupaten_kota", "nama_kabupaten_kota", "persentase_penduduk_miskin"]]
fact_abs = abs_miskin[["tahun", "kode_kabupaten_kota", "nama_kabupaten_kota", "jumlah_penduduk_miskin"]]
fact_kategori = kategori[["tahun", "periode_bulan", "kategori_daerah", "jumlah_penduduk"]]

dim_kabupaten.to_csv(PROCESSED_DIR / "dim_kabupaten.csv", index=False)
fact_pkh.to_csv(PROCESSED_DIR / "fact_pkh.csv", index=False)
fact_persen.to_csv(PROCESSED_DIR / "fact_kemiskinan_persen.csv", index=False)
fact_abs.to_csv(PROCESSED_DIR / "fact_kemiskinan_abs.csv", index=False)
fact_kategori.to_csv(PROCESSED_DIR / "fact_kemiskinan_kategori.csv", index=False)

print("Saved to:", PROCESSED_DIR)


## Quick checks
Cek rentang tahun dan jumlah baris untuk memastikan proses berhasil.

In [ ]:
def show_stats(path, year_col="tahun"):
    df = pd.read_csv(path)
    years = df[year_col].dropna().astype(int)
    print(path.name, "rows:", len(df), "years:", years.min(), "-", years.max())

show_stats(PROCESSED_DIR / "fact_pkh.csv")
show_stats(PROCESSED_DIR / "fact_kemiskinan_abs.csv")
show_stats(PROCESSED_DIR / "fact_kemiskinan_persen.csv")
show_stats(PROCESSED_DIR / "fact_kemiskinan_kategori.csv")


## Export / download hasil
Jika ingin download ke lokal:
```
from google.colab import files
files.download('/content/data/processed/fact_pkh.csv')
```


# Prediksi 5 Tahun ke Depan
Bagian ini membuat **prediksi 5 tahun** untuk PKH, kemiskinan absolut, dan kemiskinan persen.
Metode: Holt-Winters (trend additive).

In [ ]:
import numpy as np
from statsmodels.tsa.holtwinters import ExponentialSmoothing
import warnings
warnings.filterwarnings("ignore")

HORIZON = 5

def forecast_series(values, years, horizon=5):
    values = values.dropna().astype(float)
    if len(values) < 2 or values.nunique() == 1:
        return [float(values.iloc[-1])] * horizon
    # Use a proper yearly index to avoid statsmodels warnings
    series = values.copy()
    series.index = pd.PeriodIndex(years.astype(int), freq="Y")
    model = ExponentialSmoothing(series, trend="add", seasonal=None, initialization_method="estimated")
    fit = model.fit(optimized=True)
    return [float(x) for x in fit.forecast(horizon)]

def forecast_by_kabkota(df, value_col, horizon=5, clip_min=None, clip_max=None):
    results = []
    last_year = int(df["tahun"].max())
    for (kode, nama), group in df.groupby(["kode_kabupaten_kota", "nama_kabupaten_kota"]):
        group = group.sort_values("tahun")
        preds = forecast_series(group[value_col], group["tahun"], horizon)
        for i, val in enumerate(preds, start=1):
            if clip_min is not None:
                val = max(clip_min, val)
            if clip_max is not None:
                val = min(clip_max, val)
            results.append({
                "tahun": last_year + i,
                "kode_kabupaten_kota": int(kode),
                "nama_kabupaten_kota": nama,
                "value": float(val),
            })
    return pd.DataFrame(results)

pred_pkh = forecast_by_kabkota(fact_pkh, "jumlah_penerima_manfaat", HORIZON, clip_min=0.0)
pred_abs = forecast_by_kabkota(fact_abs, "jumlah_penduduk_miskin", HORIZON, clip_min=0.0)
pred_persen = forecast_by_kabkota(fact_persen, "persentase_penduduk_miskin", HORIZON, clip_min=0.0, clip_max=100.0)

pred_pkh.to_csv(PROCESSED_DIR / "pred_pkh.csv", index=False)
pred_abs.to_csv(PROCESSED_DIR / "pred_kemiskinan_abs.csv", index=False)
pred_persen.to_csv(PROCESSED_DIR / "pred_kemiskinan_persen.csv", index=False)

print("Prediksi tersimpan di", PROCESSED_DIR)


# Visualisasi Ringkas
Visualisasi sederhana untuk melihat tren historis dan prediksi.

In [ ]:
import matplotlib.pyplot as plt

def avg_by_year(df, value_col):
    return df.groupby("tahun", as_index=False)[value_col].mean()

hist_pkh = avg_by_year(fact_pkh, "jumlah_penerima_manfaat")
hist_abs = avg_by_year(fact_abs, "jumlah_penduduk_miskin")
hist_persen = avg_by_year(fact_persen, "persentase_penduduk_miskin")

pred_pkh_avg = pred_pkh.groupby("tahun", as_index=False)["value"].mean()
pred_abs_avg = pred_abs.groupby("tahun", as_index=False)["value"].mean()
pred_persen_avg = pred_persen.groupby("tahun", as_index=False)["value"].mean()

def plot_history_forecast(hist_df, pred_df, title, ylabel):
    plt.figure(figsize=(7,4))
    plt.plot(hist_df["tahun"], hist_df.iloc[:,1], marker="o", label="Aktual")
    plt.plot(pred_df["tahun"], pred_df["value"], marker="o", label="Prediksi")
    plt.title(title)
    plt.xlabel("Tahun")
    plt.ylabel(ylabel)
    plt.legend()
    plt.grid(alpha=0.2)
    plt.show()

plot_history_forecast(hist_pkh, pred_pkh_avg, "PKH (Rata-rata)", "Penerima")
plot_history_forecast(hist_abs, pred_abs_avg, "Kemiskinan Absolut (Rata-rata)", "Ribu orang")
plot_history_forecast(hist_persen, pred_persen_avg, "Kemiskinan (%) Rata-rata", "%")


# Perbandingan 3 Metode (Holt, ARIMA, Linear)
Evaluasi sederhana memakai **2 tahun terakhir** sebagai data uji.
Akurasi dirangkum dalam MAE/RMSE/MAPE dan skor 0-100 (100 - MAPE).

In [ ]:
from statsmodels.tsa.arima.model import ARIMA

TEST_YEARS = 2
METHODS = ["holt", "arima", "linear"]

def forecast_series_arima(values, years, horizon=5):
    values = values.dropna().astype(float)
    if len(values) < 3 or values.nunique() == 1:
        return [float(values.iloc[-1])] * horizon
    series = values.copy()
    series.index = pd.PeriodIndex(years.astype(int), freq="Y")
    model = ARIMA(series, order=(1, 1, 1))
    fit = model.fit()
    return [float(x) for x in fit.forecast(steps=horizon)]

def forecast_series_linear(values, years, horizon=5):
    values = values.dropna().astype(float)
    if len(values) < 2 or values.nunique() == 1:
        return [float(values.iloc[-1])] * horizon
    x = np.arange(len(values))
    coef = np.polyfit(x, values, 1)
    steps = np.arange(len(values), len(values) + horizon)
    return [float(v) for v in (coef[0] * steps + coef[1])]

def predict_series(values, years, method, horizon):
    if method == "holt":
        return forecast_series(values, years, horizon)
    if method == "arima":
        return forecast_series_arima(values, years, horizon)
    if method == "linear":
        return forecast_series_linear(values, years, horizon)
    return forecast_series(values, horizon)

def evaluate_methods(df, value_col, test_years=2, clip_min=None, clip_max=None):
    years = sorted(df["tahun"].unique())
    if len(years) <= test_years:
        return pd.DataFrame()
    train_end = years[-(test_years + 1)]
    results = []
    for method in METHODS:
        all_rmse = []
        all_mae = []
        all_mape = []
        for (_, _), group in df.groupby(["kode_kabupaten_kota", "nama_kabupaten_kota"]):
            group = group.sort_values("tahun")
            train = group[group["tahun"] <= train_end]
            test = group[group["tahun"] > train_end].head(test_years)
            if len(train) < 3 or len(test) < test_years:
                continue
            actual = test[value_col].astype(float).values
            preds = np.array(predict_series(train[value_col], train["tahun"], method, test_years))
            if clip_min is not None:
                preds = np.maximum(preds, clip_min)
            if clip_max is not None:
                preds = np.minimum(preds, clip_max)
            diff = preds - actual
            rmse = np.sqrt(np.mean(diff**2))
            mae = np.mean(np.abs(diff))
            denom = np.where(actual == 0, 1.0, actual)
            mape = np.mean(np.abs(diff) / denom) * 100
            all_rmse.append(rmse)
            all_mae.append(mae)
            all_mape.append(mape)
        if all_rmse:
            rmse_mean = float(np.mean(all_rmse))
            mae_mean = float(np.mean(all_mae))
            mape_mean = float(np.mean(all_mape))
            score = max(0.0, 100.0 - mape_mean)
            results.append({
                "method": method,
                "rmse": rmse_mean,
                "mae": mae_mean,
                "mape": mape_mean,
                "score": score,
            })
    return pd.DataFrame(results).sort_values("score", ascending=False)

compare_pkh = evaluate_methods(fact_pkh, "jumlah_penerima_manfaat", TEST_YEARS, clip_min=0.0)
compare_abs = evaluate_methods(fact_abs, "jumlah_penduduk_miskin", TEST_YEARS, clip_min=0.0)
compare_persen = evaluate_methods(fact_persen, "persentase_penduduk_miskin", TEST_YEARS, clip_min=0.0, clip_max=100.0)

print("PKH:\n", compare_pkh)
print("Kemiskinan_abs:\n", compare_abs)
print("Kemiskinan%:\n", compare_persen)


# Visualisasi per Kab/Kota (Top 10)
Visualisasi hasil prediksi pada **tahun terakhir** untuk 10 kab/kota dengan nilai tertinggi.

In [ ]:
def plot_top_kabkota(pred_df, title, unit_label, top_n=10):
    last_year = pred_df["tahun"].max()
    top = pred_df[pred_df["tahun"] == last_year].sort_values("value", ascending=False).head(top_n)
    plt.figure(figsize=(7,4))
    plt.barh(top["nama_kabupaten_kota"], top["value"])
    plt.gca().invert_yaxis()
    plt.title(f"{title} (Top {top_n}) - Tahun {int(last_year)}")
    plt.xlabel(unit_label)
    plt.tight_layout()
    plt.show()

plot_top_kabkota(pred_pkh, "Prediksi PKH", "Penerima")
plot_top_kabkota(pred_abs, "Prediksi Kemiskinan Absolut", "Ribu orang")
plot_top_kabkota(pred_persen, "Prediksi Kemiskinan (%)", "%")
